In [24]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from nostrategy import GS2D
import torch
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


device = 'cuda'
gs = GS2D(img_size=(256, 256, 3), device=device)


# Function to use the slider value and display the result
def update_image(_):
    rgb_color = mcolors.hex2color(color_picker.value)
    params_splat0 = [sx.value, sy.value, rho_.value, m_x.value, m_y.value, rgb_color[0], rgb_color[1], rgb_color[2], a.value]

    w = torch.tensor([params_splat0], device=device)
    params_label.value = f'Params:<br />{params_splat0}'

    # Parse parameters
    sigma, rho, mean, color, alpha = gs.parse_param(w)

    pparams_tlist = [sigma, rho, mean, color, alpha]
    pparams_br = [str(tensor.tolist()) for tensor in pparams_tlist]
    params_parse_label.value = f'Params parsed:<br />{"<br />".join(pparams_br)}'

    # Draw the Gaussians
    img = gs.draw_gaussian(sigma, rho, mean, color, alpha)
    imgcpu = img.permute(1, 2, 0).cpu().detach().numpy()

    with out:
        clear_output(wait=True)
        plt.imshow(imgcpu)
        plt.axis('off')
        plt.show()


sx = widgets.FloatSlider(value=-2, min=-3, max=3, step=0.01, description='sx:')
sy = widgets.FloatSlider(value=-3, min=-3, max=3, step=0.01, description='sy:')
rho_ = widgets.FloatSlider(value=.5, min=-1, max=1, step=0.01, description='rho:')
m_x = widgets.FloatSlider(value=.5, min=-2, max=2, step=0.01, description='m_x:')
m_y = widgets.FloatSlider(value=-1, min=-2, max=2, step=0.01, description='m_y:')
color_picker = widgets.ColorPicker(value='#ff0000', description='Color:')
a = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='a:')

params_ui = [sx, sy, rho_, m_x, m_y, color_picker, a]
for widget in params_ui:
    widget.observe(update_image, names='value')


update_button = widgets.Button(description="Update Image")
update_button.on_click(update_image)

# Create a label widget to display the result
params_label = widgets.HTML(value='Params:')
params_parse_label = widgets.HTML(value=f'Parsed Params\n:')

out = widgets.Output()

# Display the slider and result label
sliders = widgets.VBox(params_ui)
col2 = widgets.VBox([params_label, params_parse_label])
ui = widgets.HBox([sliders, out, col2])
display(ui)
update_image(None)